# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @luca_salimon
  
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [5]:
#Produto escolhido:
produto = 'brahma'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [8]:
teste = pd.read_excel("brahma.xlsx")
teste_rel = pd.DataFrame(teste[teste.Classificação == 'r'])
teste_irel = pd.DataFrame(teste[teste.Classificação == 'i'])

In [9]:

teste.Treinamento = teste.Treinamento.replace(',','',regex = True)

teste_rel.Treinamento = teste_rel.Treinamento.replace(',','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('@','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace(';','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/8niv5gknsl','',regex=True) 
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/8niv5gknsl','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/rn9fknwvj5','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/knnvlsekmg','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/pgv7b8g1aq https://t.co/x1oexhegtz','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/nvlxmbdbik','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/yiwslxwgwz','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('\[','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('\]','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('/','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('https://t.co/pckxnvyavh','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('rt','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('\?','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('\!','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace(':','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('_','',regex=True)
teste_rel.Treinamento = teste_rel.Treinamento.replace('\n','',regex=True)


teste_irel.Treinamento = teste_irel.Treinamento.replace(',','',regex=True) 
teste_irel.Treinamento = teste_irel.Treinamento.replace('@','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace(';','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/8niv5gknsl','',regex=True) 
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/8niv5gknsl','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/rn9fknwvj5','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/knnvlsekmg','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/pgv7b8g1aq https://t.co/x1oexhegtz','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/nvlxmbdbik','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/yiwslxwgwz','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('\[','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('\]','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('/','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('https://t.co/pckxnvyavh','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('rt','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('\?','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('\!','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace(':','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('_','',regex=True)
teste_irel.Treinamento = teste_irel.Treinamento.replace('\n','',regex=True)




In [10]:
import numpy as np
coluna_Treinamento = pd.Series(np.sum(teste.Treinamento + " ").split())
coluna_Classificação = pd.Series(np.sum(teste.Classificação + " ").split())
num_palavras = len(coluna_Treinamento.value_counts())
#num_palavras
#coluna_Treinamento.count('brahma')


In [11]:
#teste_irel

In [12]:
palavras_rel = np.sum(teste_rel.Treinamento + " ").split()
palavras_rel_2 = pd.Series(np.sum(teste_rel.Treinamento + " ").split())
palavras_irel = np.sum(teste_irel.Treinamento + " ").split()
palavras_irel_2= pd.Series(np.sum(teste_irel.Treinamento + " ").split())
numero_rel = len(palavras_rel_2.value_counts())
numero_irel = len(palavras_irel_2.value_counts())
lista_rel = palavras_rel_2.value_counts()
lista_irel = palavras_irel_2.value_counts()

In [13]:
relevante = pd.Series(np.sum(teste_rel.Classificação + " ").split())
relevante_count =relevante.value_counts()
#print(relevante_count)
irrelevante = pd.Series(np.sum(teste_irel.Classificação + " ").split())
irrelevante_count = irrelevante.value_counts()
#print(irrelevante_count)
prob_rel = relevante_count/300
#print(prob_rel)
prob_irel = irrelevante_count/300
#print(prob_irel)



In [14]:
def funcao(a):
    tweet = a.split(' ')
    probab_rel = []
    probab_irel = []
    for i in range(len(tweet)):
        probab_rel.append((palavras_rel.count(tweet[i])+1)/numero_rel+len(tweet))
        probab_irel.append((palavras_irel.count(tweet[i])+1)/numero_irel + len(tweet))
    prob_rel_tot = np.prod(probab_rel)
    prob_irel_tot = np.prod(probab_irel)
    if prob_rel_tot > prob_irel_tot:
        return 'Relevante'
    else:
        return 'Irrelevante'
    
a = "rt @_luizamoreiira: desculpa brahma, ela não sabe o que diz https://t.co/8niv5gknsl"
        
print(funcao(a))

Irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [24]:
teste_2 = pd.read_excel('brahma.xlsx',sheetname = 'Teste',sep = ',')
teste_2['programa'] = ''


teste_2.Teste = teste_2.Teste.replace(',','',regex=True)
teste_2.Teste = teste_2.Teste.replace('@','',regex=True)
teste_2.Teste = teste_2.Teste.replace(';','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/8niv5gknsl','',regex=True) 
teste_2.Teste = teste_2.Teste.replace('https://t.co/8niv5gknsl','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/rn9fknwvj5','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/knnvlsekmg','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/pgv7b8g1aq https://t.co/x1oexhegtz','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/nvlxmbdbik','',regex=True)
teste_2.Teste = teste_2.Teste.replace('https://t.co/yiwslxwgwz','',regex=True)
teste_2.Teste = teste_2.Teste.replace('\[','',regex=True)
teste_2.Teste = teste_2.Teste.replace('\]','',regex=True)
teste_2.Teste = teste_2.Teste.replace('/','',regex=True)
teste_2.Teste= teste_2.Teste.replace('https://t.co/pckxnvyavh','',regex=True)
teste_2.Teste = teste_2.Teste.replace('rt','',regex=True)
teste_2.Teste = teste_2.Teste.replace('\?','',regex=True)
teste_2.Teste = teste_2.Teste.replace('\!','',regex=True)
teste_2.Teste = teste_2.Teste.replace(':','',regex=True)
teste_2.Teste = teste_2.Teste.replace('_','',regex=True)
teste_2.Teste = teste_2.Teste.replace('\n','',regex=True)

teste_2


,Teste,Unnamed: 1,classificação,programa
0,loupfvr desejando uma praia e uma brahma gela...,NaN,r,
1,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,
2,squechrj só uma brahma bem gelada agora 😏,NaN,r,
3,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,
4,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,
5,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,
6,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,
7,agora a brahma tá fazendo propaganda pra dizer...,NaN,i,
8,"gabriellesiiva só umas brahma bem gelada ""mrm...",NaN,r,
9,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,


In [27]:
run = []

for i in range(len(teste_2.Teste)):
    run.append(funcao(teste_2.Teste[i]))

In [28]:
teste_2['programa'] = pd.Series(run)
teste_2

,Teste,Unnamed: 1,classificação,programa
0,loupfvr desejando uma praia e uma brahma gela...,NaN,r,Relevante
1,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante
2,squechrj só uma brahma bem gelada agora 😏,NaN,r,Relevante
3,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante
4,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante
5,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante
6,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante
7,agora a brahma tá fazendo propaganda pra dizer...,NaN,i,Irrelevante
8,"gabriellesiiva só umas brahma bem gelada ""mrm...",NaN,r,Relevante
9,luizamoreiira desculpa brahma ela não sabe o ...,NaN,i,Irrelevante


In [39]:
resultado =[]
for o in range(len(teste_2.Teste)):
    if teste_2.classificação[o] == 'r':
        if teste_2.programa[o] == 'Relevante':
            resultado.append('verdadeiro positivo')
        if teste_2.programa[o] == 'Irrelevante':
            resultado.append('falso negativo')
            
    elif teste_2.classificação[o] == 'i':
        if teste_2.programa[o] == 'Irrelevante':
            resultado.append('verdadeiro negativo')
        if teste_2.programa[o] == 'Relevante':
            resultado.append('falso positivo')
acertos = pd.Series(resultado)
acertos.value_counts()

verdadeiro negativo    103
verdadeiro positivo     49
falso negativo          42
falso positivo           6
dtype: int64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


In [37]:
resultado =[]
for o in range(len(teste_2.Teste)):
    if teste_2.classificação[o] == 'r':
        if teste_2.programa[o] == 'Relevante':
            resultado.append('verdadeiro positivo')
        if teste_2.programa[o] == 'Irrelevante':
            resultado.append('falso negativo')
            
    elif teste_2.classificação[o] == 'i':
        if teste_2.programa[o] == 'Irrelevante':
            resultado.append('verdadeiro negativo')
        if teste_2.programa[o] == 'Relevante':
            resultado.append('falso positivo')
acertos = pd.Series(resultado)
acertos.value_counts()

verdadeiro negativo    103
verdadeiro positivo     49
falso negativo          42
falso positivo           6
dtype: int64

In [43]:
por_vn = resultado.count('verdadeiro negativo')/2
por_vp = resultado.count('verdadeiro positivo')/2
por_fn = resultado.count('falso negativo')/2
por_fp = resultado.count('falso positivo')/2

print('porcentagem de verdadeiro negativo: {0}%\nporcentagem de verdadeiro positivo: {1}%\nporcentagem de falso negativo: {2}\nporcentagem de falso positivo: {3}%'.format(por_vn,por_vp,por_fn,por_fp))

porcentagem de verdadeiro negativo: 51.5%
porcentagem de verdadeiro positivo: 24.5%
porcentagem de falso negativo: 21.0
porcentagem de falso positivo: 3.0%


## Conclusão

O nosso programa teve uma quantidade alta de acerto, 76% de acerto, porque estavamos colentando frases, relevantes sendo boas ou ruins para marca, assim as palavras mesmo que se repetissem na frase a favor e contra a marca o classificador entende como relevante. O erro se da principalmente pela contagem de artigos e conjunções que aparecem em quase todas as frases e por causa da base de dados que por nao ser tão ampla e ter diversas repetições abre uma margem de erro.